In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy
import cv2

import torch
import torchvision
from torchvision import models
import torch.nn as nn
from torchvision import transforms,datasets
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset,ConcatDataset
from PIL import Image
import torch.nn.functional as F
from torch.nn.modules.pooling import AvgPool3d
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from itertools import product

In [2]:
torch.cuda.is_available()

True

In [3]:
train=pd.read_csv('../input/train.csv')
sample=pd.read_csv('../input/sample_submission.csv')

In [4]:
train.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17500 entries, 0 to 17499
Data columns (total 2 columns):
id            17500 non-null object
has_cactus    17500 non-null int64
dtypes: int64(1), object(1)
memory usage: 273.5+ KB


In [6]:
train['has_cactus'].value_counts()

1    13136
0     4364
Name: has_cactus, dtype: int64

In [7]:
"""extra=train[train.has_cactus==0]
train=pd.concat([train,extra],axis=0)"""

'extra=train[train.has_cactus==0]\ntrain=pd.concat([train,extra],axis=0)'

# Image Transformation

In [8]:
image_transforms={
    'train':transforms.Compose([
        transforms.RandomRotation(degrees=0),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],
                            [0.229,0.224,0.225])]),
    'test':transforms.Compose([
        transforms.RandomRotation(degrees=0),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],
                            [0.229,0.224,0.225])
    ])
}

# Load The Data

In [9]:
train_set,val_set=train_test_split(train,stratify=train.has_cactus,test_size=0.2)

len1=len(train_set)
len2=len(val_set)

# Loading Data From Folders
train_dir='train'
test_dir='test'

In [10]:
class dataset_(torch.utils.data.Dataset):
    def __init__(self,labels,data_directory,transform):
        super().__init__()

        #characterizes a dataset for Pytorch
        
        self.list_id=labels.values[:,0]
        self.labels=labels.values[:,1]
        self.data_dir=data_directory
        self.transform=transform
    
    def __len__(self):
        # Denotes the tota number of samples
        return len(self.list_id)
    
    def __getitem__(self,index):
        name=self.list_id[index]
        img=Image.open('../input/train/train/{}'.format(name))
        img=self.transform(img)
        return img,torch.tensor(self.labels[index],dtype=torch.float32)

In [11]:
train_set=dataset_(train_set,train_dir,image_transforms['train'])
#train_df=DataLoader(train_df,batch_size=120,shuffle=True)

val_set=dataset_(val_set,train_dir,image_transforms['test'])
#val_df=DataLoader(val_df,batch_size=120,shuffle=True)

In [12]:
lst,labels=next(iter(train_set))

In [13]:
lst.dtype,labels.dtype,labels

(torch.float32, torch.float32, tensor(0.))

# Creating A Model

In [14]:
def size(image_size,ker,stri,pad=0):
    return (image_size-ker+2*pad)/stri +1

In [15]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=100,kernel_size=3)
        self.conv2=nn.Conv2d(in_channels=100,out_channels=150,kernel_size=3)
        self.conv3=nn.Conv2d(in_channels=150,out_channels=250,kernel_size=3)
        #self.conv4=nn.Conv2d(in_channels=16,out_channels=20,kernel_size=3)

        self.d1=nn.Dropout(0.3)
        self.fc1=nn.Linear(in_features=250*4*4,out_features=500)
        self.fc2=nn.Linear(in_features=500,out_features=200)
        self.out=nn.Linear(in_features=200,out_features=2)


    def forward(self,t):
        #input layer
        t=t
        #first convolutional layer
        t=F.max_pool2d(F.relu(self.conv1(t)),kernel_size=2,stride=2)
        
        #Second Convolutional Layer
        
        t=F.max_pool2d(F.relu(self.conv2(t)),stride=1,kernel_size=3)
        
        #Third Convolution Layer
        
        t=F.max_pool2d(F.relu(self.conv3(t)),stride=2,kernel_size=3)
        
        #Fourth Convolutional Layer
        #t=F.max_pool2d(F.relu(self.conv4(t)),stride=3,kernel_size=3)
        
        # First linear Layer
        t=t.reshape(-1,250*4*4)
        t=F.relu(self.fc1(t))
        t=self.d1(t)
        
        #Second Linear Layer
        t=F.relu(self.fc2(t))
        t=self.d1(t)
        
        #Third Linear Layer
        t=self.out(t)
        
        return t

# Training Process

In [16]:
batch_sizes=130
lrs=0.1
train_loss=[]
val_loss=[]
train_correct=[]
val_correct=[]
epoch=[]
model=Model()
model=model.to('cuda:0')
optimizer=optim.SGD(model.parameters(),lr=lrs)
    
train_df=DataLoader(train_set,batch_size=batch_sizes,shuffle=True)
val_df=DataLoader(val_set,batch_size=batch_sizes,shuffle=True)

for i in range(70):
    total_loss=0 # Total loss for an epoch
    total_correct=0 #Total correct number of predictions
    for batch in train_df:
        images,labels=batch #loading a batch

        images=images.to('cuda:0') #changing the device
        labels=labels.to('cuda:0') 

        preds=model(images) #predicting the probabilities of being in a class
        labels=labels.long() #changing data type
        f=nn.Sigmoid()
        #preds=f(preds)

        loss=F.cross_entropy(preds,labels) #CALCULATING lOSS
            #changing the earlier gradient

        optimizer.zero_grad()

        loss.backward() # Calculating the gradient
        optimizer.step()# Updating the gradient
        total_loss+=loss.item() # calculating the total loss 
        total_correct+=preds.argmax(dim=1).eq(labels).sum().item()#Calculating the correct prediction in a single epoch
        del images,labels
        
    train_loss.append(total_loss)
    train_correct.append(total_correct/len1)
    epoch.append(i+1)

    with torch.no_grad():
        total_val_loss=0
        total_val_correct=0
        for val_batch in val_df:
            val_im,val_lab=val_batch
            val_im=val_im.to('cuda:0')
            val_lab=val_lab.to('cuda:0')

            val_preds=model(val_im)
            val_lab=val_lab.long()
            #val_preds=f(val_preds)
            
            loss_val=F.cross_entropy(val_preds,val_lab)
            total_val_loss+=loss_val
            total_val_correct+=val_preds.argmax(dim=1).eq(val_lab).sum().item()

        val_loss.append(total_val_loss)
        val_correct.append(total_val_correct/len2)

        print("Epoch {}\t train_loss {} \t val_loss {}\n".format(epoch[i],total_loss,total_val_loss))

Epoch 1	 train_loss 37.909471310675144 	 val_loss 5.014248371124268

Epoch 2	 train_loss 17.54711987823248 	 val_loss 5.381856918334961

Epoch 3	 train_loss 11.734003443270922 	 val_loss 2.42157244682312

Epoch 4	 train_loss 9.560369810089469 	 val_loss 1.8283510208129883

Epoch 5	 train_loss 6.624355269595981 	 val_loss 1.8180549144744873

Epoch 6	 train_loss 6.935572700574994 	 val_loss 1.418682336807251

Epoch 7	 train_loss 5.137770913541317 	 val_loss 1.3786144256591797

Epoch 8	 train_loss 4.154282038565725 	 val_loss 1.4015976190567017

Epoch 9	 train_loss 3.796827021986246 	 val_loss 1.113948941230774

Epoch 10	 train_loss 3.5157620566897094 	 val_loss 1.1843180656433105

Epoch 11	 train_loss 3.5170694766566157 	 val_loss 1.1555947065353394

Epoch 12	 train_loss 2.9657405376201496 	 val_loss 1.1343905925750732

Epoch 13	 train_loss 2.302657369989902 	 val_loss 0.9347913861274719

Epoch 14	 train_loss 2.7122887826990336 	 val_loss 0.8577979803085327

Epoch 15	 train_loss 2.148860

In [17]:
val_correct[-5:],train_correct[-5:]

([0.9937142857142857,
  0.9922857142857143,
  0.9931428571428571,
  0.9925714285714285,
  0.9942857142857143],
 [0.9995714285714286, 0.9987857142857143, 1.0, 1.0, 1.0])

In [18]:
trans=transforms.ToTensor()
submissions=pd.DataFrame({'id':[],'has_cactus':[]})

In [19]:
with torch.no_grad():
    for i in range(4000):
        im=Image.open('../input/test/test/{}'.format(sample.iloc[i][0]))
        im=image_transforms['test'](im)
        im=im.to('cuda:0')
        im=im.unsqueeze(dim=0)
        test_pred=model(im)
        #test_pred=f(test_pred)
        test_pred=test_pred.argmax(dim=1).sum().item()
        submissions=pd.concat([submissions,pd.DataFrame({'id':[sample.iloc[i][0]],
                                                                 'has_cactus':[test_pred]})])

In [20]:
submissions['has_cactus'].value_counts()

1.0    2904
0.0    1096
Name: has_cactus, dtype: int64

In [21]:
submissions.to_csv('samplesubmission.csv',index=False)